In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib
%matplotlib inline
import os
import pickle

In [3]:
def undistort(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(Left_lane.objpoints, Left_lane.imgpoints, gray.shape[::-1],None,None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [4]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [5]:
def calibrate_camera(calibration_folder):

    #Initialize object and image points
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = []
    images = glob.glob(calibration_folder)
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

        if ret == True:
            imgpoints.append(corners)
            objpoints.append(objp)
    return  objpoints, imgpoints

In [6]:
def magnitude_thresh(img):
    img = np.copy(img)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = gaussian_blur(gray, 5)
    
    #magnitude
    abs_sobel_x = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=11))
    abs_sobel_y = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=11))
    abs_sobelxy = np.sqrt((abs_sobel_x * abs_sobel_x)+(abs_sobel_y * abs_sobel_y))
    scaled_mag = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))
    mag_binary = np.zeros_like(scaled_mag)
    mag_binary[(scaled_mag >= 60) & (scaled_mag <=255)] = 1
    
    return mag_binary

In [7]:
def direction_thresh(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=11)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=11)
    
    abs_grad_dir = np.arctan2(np.absolute(sobel_y), np.absolute(sobel_x))
    dir_binary =  np.zeros_like(abs_grad_dir)
    dir_binary[(abs_grad_dir >= .7) & (abs_grad_dir <= 1.3)] = 1
                                          
    return dir_binary

In [8]:
def channel_threshold_LUV(img,threshold_range):
    
    l_channel_luv = cv2.cvtColor(img, cv2.COLOR_BGR2LUV)[:,:,0]

    
    l_binary = np.zeros_like(l_channel_luv)
    l_binary[(l_channel_luv >= threshold_range[0]) & (l_channel_luv <= threshold_range[1])] = 1
    
    return l_binary

In [9]:
def channel_threshold_YCrCb(img,threshold_range):
    brightYCB = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)[:,:,0]
    
    ycrcb_binary = np.zeros_like(brightYCB)
    ycrcb_binary[(brightYCB >= threshold_range[0]) & (brightYCB <= threshold_range[1])] = 1
    
    return ycrcb_binary

In [10]:
def channel_threshold_LAB(img,threshold_range):
    
    b_channel_lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)[:,:,2]
        
    b_binary = np.zeros_like(b_channel_lab)
    b_binary[(b_channel_lab >= threshold_range[0]) & (b_channel_lab <= threshold_range[1])] = 1
    
    return b_binary

In [11]:
def sobel_x(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    
    abs_sobelx = np.absolute(sobel_x)
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    sobel_x_binary = np.zeros_like(scaled_sobelx)
    sobel_x_binary[(scaled_sobelx >= 15) & (scaled_sobelx <= 255)] = 1
    
    return sobel_x_binary


def sobel_y(img):  
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    
    abs_sobely = np.absolute(sobel_y)
    scaled_sobely = np.uint8(255*abs_sobely/np.max(abs_sobely))
    sobel_y_binary = np.zeros_like(scaled_sobely)
    sobel_y_binary[(scaled_sobely >= 35) & (scaled_sobely <= 255)] = 1
    
    return sobel_y_binary

In [12]:
def color_threshold(img):
    
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel > 60) & (s_channel <= 255)] = 1
    
    
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >120) & (v_channel <= 255)] = 1 
    
    
    color_binary = np.zeros_like(s_channel)
    color_binary[(s_binary == 1) & (v_binary == 1)] = 1
    return color_binary

In [13]:
def threshold_video_hard(img):
    sobel_x_binary = sobel_x(img)
    sobel_y_binary = sobel_y(img)
    
    mag_binary = magnitude_thresh(img)
    dir_binary = direction_thresh(img)
    
    color_binary = color_threshold(img)
    
    result_lab = channel_threshold_LAB(img, (160,215))
    result_luv = channel_threshold_LUV(img, (215,255))
    
    hard_output = np.zeros_like(sobel_x_binary)
    hard_output[(sobel_x_binary == 1) & (sobel_y_binary == 1) & (mag_binary == 1) | (result_lab == 1) | (result_luv == 1) | (color_binary == 1) | (mag_binary == 1) & (dir_binary == 1)] = 1
    return hard_output

In [14]:
def transform_prespective(img):
    img_size = (img.shape[1], img.shape[0])
    src1 = (590, 450)
    scr2 = (180, img.shape[0])
    src3 = (1130, img.shape[0])
    src4 = (690, 450)
    src_points = np.array([[src1, scr2, src3, src4]]).astype('float32')
    
    dst1 = (240, 0)         
    dst2 = (240, img.shape[0])
    dst3 = (1040, img.shape[0])
    dst4 = (1040, 0)
    dst_points = np.array([[dst1, dst2, dst3, dst4]]).astype('float32')
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    Minv = cv2.getPerspectiveTransform(dst_points, src_points)
    return cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR),Minv

In [15]:
def find_lane_pixels_vid(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 10
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 2

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    return leftx, lefty, rightx, righty,leftx_current,rightx_current, out_img

In [16]:
def fit_polynomial_vid(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty,leftx_current,rightx_current, out_img = find_lane_pixels_vid(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    

    return out_img, left_fit, right_fit,left_fitx,right_fitx,leftx_current,rightx_current

In [17]:
def find_center_offset(binary_warped,left_fit,right_fit):
    
    bottom_y = binary_warped.shape[0] - 1
    bottom_x_left = left_fit[0]*(bottom_y**2) + left_fit[1]*bottom_y + left_fit[2]
    bottom_x_right = right_fit[0]*(bottom_y**2) + right_fit[1]*bottom_y + right_fit[2]
    offset = binary_warped.shape[1]/2 - (bottom_x_left + bottom_x_right)/2

    #pixel meter converstion
    offset *= 3.7/700
    
    return offset

In [18]:
def curve_radius(y_max):

    # meters space conversion
    y_pixel2m = 30/720
    x_pixel2m = 3.7/700

    
    leftx = Left_lane.pos_x
    lefty = Left_lane.pos_y
    rightx = Right_lane.pos_x
    righty = Right_lane.pos_y

    # Fit new polynomials to x,y in world space
    left_curve_m = np.polyfit(lefty*y_pixel2m, leftx*y_pixel2m, 2)
    right_curve_m = np.polyfit(righty*x_pixel2m, rightx*x_pixel2m, 2)
    
    # Calculate the new radius for the curve
    left_curve_radius = ((1 + (2*left_curve_m[0]*y_max*y_pixel2m + left_curve_m[1])**2)**1.5) / np.absolute(2*left_curve_m[0])
    right_curve_radius = ((1 + (2*right_curve_m[0]*y_max*y_pixel2m + right_curve_m[1])**2)**1.5) / np.absolute(2*right_curve_m[0])


    return left_curve_radius, right_curve_radius

In [19]:
def Lane_Area_Vid(binary_warped, undistorted, left_fit, right_fit, m_inv,avg_radius):


    leftx = Left_lane.pos_x
    lefty = Left_lane.pos_y
    rightx = Right_lane.pos_x
    righty = Right_lane.pos_y
    
    #find offset
    center_offset = find_center_offset(binary_warped, left_fit,right_fit)
    
    #create lane area mask
    temp_lane_area = np.zeros_like(undistorted)

    # Recast the x and y points into usable format for cv2.fillPoly()
    ploty = np.linspace(0, undistorted.shape[0]-1, undistorted.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

     #fill green color
    cv2.fillPoly(temp_lane_area, np.int_([pts]), (0,255, 0))

    # Highlight lane area mask in undistorted
    inverse_transformed = cv2.warpPerspective(temp_lane_area, m_inv, (undistorted.shape[1], undistorted.shape[0]))
    
    # Combine the result with the original image
    lane_area = cv2.addWeighted(undistorted, 1, inverse_transformed, 0.3, 0)

    # Write text in image
    radius_text = 'Radius of curve is ' + str(round(avg_radius, 2)) + ' m' 
    offset_text = 'Offset from lane center:'+ str(abs(round(center_offset, 2)))
    
                      
    lane_area = cv2.putText(lane_area, radius_text, (450,40), 0, 1, (0,0,0), 2, cv2.LINE_AA)
    lane_area = cv2.putText(lane_area, offset_text, (450,70), 0, 1, (0,0,0), 2, cv2.LINE_AA)

    return lane_area

In [20]:
def set_lane_pixel(binary_warped):
    
    leftx, lefty, rightx, righty,leftx_current,rightx_current, out_img = find_lane_pixels_vid(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    
    #Setting lane pixel positions so that could be used by other functions as well
    Left_lane.pos_x = leftx
    Left_lane.pos_y = lefty
    Right_lane.pos_x = rightx
    Right_lane.pos_y = righty
    
    return left_fit, right_fit, leftx, rightx,lefty,righty

In [21]:
class Line():
    
    def __init__(self):
        self.pos_x = None # fit values position in xaxis
        self.pos_y = None # fit values position in yaxis
        calibration_folder ='camera_cal/calibration*.jpg'
        self.objpoints, self.imgpoints = calibrate_camera(calibration_folder)
        
        
#         self.detected = False  
#         # x values of the last n fits of the line
#         self.recent_xfitted = [] 
#         #average x values of the fitted line over the last n iterations
#         self.bestx = None     
#         #polynomial coefficients averaged over the last n iterations
#         self.best_fit = None  
#         #polynomial coefficients for the most recent fit
#         self.current_fit = [np.array([False])]  
#         #radius of curvature of the line in some units
#         self.radius_of_curvature = None 
#         #distance in meters of vehicle center from the line
#         self.line_base_pos = None 
#         #difference in fit coefficients between last and new fits
#         self.diffs = np.array([0,0,0], dtype='float') 
#         #x values for detected line pixels
#         self.allx = None  
#         #y values for detected line pixels
#         self.ally = None  
        
        

In [22]:

#to initialize values and later pass through for next frame
Left_lane = Line()
Right_lane = Line()


In [23]:
def process_image(img):
        
        undistorted = undistort(img)
        thresholded = threshold_video_hard(undistorted)
        warped_image, m_inv = transform_prespective(thresholded)
        
        #set values
        left_fit, right_fit, leftx, rightx,lefty,righty = set_lane_pixel(warped_image)   
        
        #find curvature
        left_curve_radius, right_curve_radius = curve_radius(undistorted.shape[0]-1 )
        avg_radius = (left_curve_radius + right_curve_radius)/2
        
        #fill lane area
        out_img = Lane_Area_Vid(warped_image, undistorted, left_fit, right_fit, m_inv,avg_radius)
        
        return out_img

In [24]:
from moviepy.editor import VideoFileClip

video_output = 'challenge_op.mp4'
clip = VideoFileClip('challenge_video.mp4')

output_clip = clip.fl_image(process_image)
%time output_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video challenge_op.mp4
[MoviePy] Writing video challenge_op.mp4


100%|████████████████████████████████████████████████████████████████████████████████| 485/485 [11:33<00:00,  1.58s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_op.mp4 

Wall time: 11min 34s


In [24]:
clip.reader.close()
clip.audio.reader.close_proc()